#### We are going to use Flickr8-dataset. The dataset is available on kaggle here: https://www.kaggle.com/datasets/adityajn105/flickr8k?resource=download

#### **Preprocess data**

In [1]:
import os
import numpy as np
from PIL import Image, UnidentifiedImageError
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import preprocess_input
from tqdm import tqdm  

print("Current Working Directory:", os.getcwd())

# Path to the Flickr8k images and captions
image_folder = "Images"  # CHANGE HERE TO YOUR OWN PATH!
caption_file = "captions.txt"  # SAME HERE!


def load_captions(filename):
    with open(filename, 'r') as file:
        text = file.read()
    captions = {}
    for line in text.strip().split('\n'):
        tokens = line.split()
        image_id, image_desc = tokens[0], tokens[1:]
        image_id = image_id.split('.')[0]
        image_desc = ' '.join(image_desc)
        if image_id not in captions:
            captions[image_id] = []
        captions[image_id].append(image_desc)
    return captions

captions = load_captions(caption_file)


# Load images
def load_images(image_folder, captions):
    images = {}
    # Use tqdm for progress bar
    for image_id in tqdm(captions, desc="Loading Images"):
        image_path = os.path.join(image_folder, image_id + '.jpg')
        try:
            image = Image.open(image_path)
            images[image_id] = np.array(image)
        except (FileNotFoundError, UnidentifiedImageError) as e:
            print(f"Error loading image '{image_path}':")
            print(f"{e}")
            continue  # Skip this image and continue with the next one
    return images


images = load_images(image_folder, captions)

tokenizer = Tokenizer(num_words=10000, oov_token="<unk>")
all_captions = [caption for caption_list in captions.values() for caption in caption_list]
tokenizer.fit_on_texts(all_captions)
captions_seq = {image_id: tokenizer.texts_to_sequences(captions_list) for image_id, captions_list in captions.items()}

2024-05-08 17:51:12.296073: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Current Working Directory: /Users/qasim/Flickr8_data


Loading Images:   0%|                                                                        | 0/8092 [00:00<?, ?it/s]

Error loading image 'Images/image,caption.jpg':
[Errno 2] No such file or directory: 'Images/image,caption.jpg'


Loading Images: 100%|█████████████████████████████████████████████████████████████| 8092/8092 [01:58<00:00, 68.13it/s]


#### **Here we are going to create a model. The model has an input shape of 224,224,3, and will use the weights from imagenet.**

#### **This model is then connected to a LSTM, for image captioning.**



In [8]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Concatenate
from tensorflow.keras.applications import VGG16

def create_model(vocab_size, max_length):
    
    vgg = VGG16(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
    vgg.trainable = False  
    image_input = Input(shape=(224, 224, 3))
    features = vgg(image_input)
    features = tf.keras.layers.GlobalAveragePooling2D()(features)
    image_dense = Dense(256, activation='relu')(features)
    
    # Caption model
    caption_input = Input(shape=(max_length,))
    caption_embedding = Embedding(input_dim=vocab_size, output_dim=256)(caption_input)
    caption_lstm = LSTM(256)(caption_embedding)
    
    # Combine image and caption information
    decoder_input = Concatenate()([image_dense, caption_lstm])
    outputs = Dense(vocab_size, activation='softmax')(decoder_input)
    
    # Final model
    model = Model(inputs=[image_input, caption_input], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model


In [10]:
model = create_model(vocab_size, max_length)
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ vgg16 (Functional)  │ (None, 7, 7, 512) │ 14,714,688 │ input_layer_4[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_5       │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 512)       │          0 │ vgg16[0][0]       │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 5, 256)    │  2,153,984 │ input_layer_5[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │    131,328 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 256)       │    525,312 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 512)       │          0 │ dense_2[0][0],    │
│ (Concatenate)       │                   │            │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 8414)      │  4,316,382 │ concatenate_2[0]… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 21,841,694 (83.32 MB)

 Trainable params: 7,127,006 (27.19 MB)

 Non-trainable params: 14,714,688 (56.13 MB)